# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door alle docker containers te starten die gedefinieerd worden in de yaml-file.
Dit kan via de docker desktop gui of via de command line door middel van het docker-compose commando.
Stoppen kan dan via dezelfde methodes

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [InsecureClient in hdfscli](https://hdfscli.readthedocs.io/en/latest/quickstart.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [1]:
from hdfs import InsecureClient
client = InsecureClient("http://localhost:9870", user='bigdata')

In [6]:
# met status kan je gaan controleren of een file/folder bestaat
client.status('/user/bigdata/HDFS/davinci_notebooks.txt', strict=False)
fs_status = client.status('/user/bigdata/HDFS/davinci_notebooks2.txt', strict=False)
if fs_status is None:
    print('None')
else:
    print(fs_status)

None


### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [11]:
# test = bestaat deze file = status
!hdfs dfs -test -d /user/bigdata/HDFS/davinci_notebooks.txt
!hdfs dfs -mkdir -p /user/bigdata/HDFS2

# files aanmaken
# put is schrijven
!hadoop fs -put ulysses.txt /user/bigdata/HDFS2/cli.txt 
# get is lezen
!hadoop fs -get /user/bigdata/HDFS2/cli.txt ulysses2.txt

In [24]:
# idem als hierboven maar in python code
if client.status('/user/bigdata/HDFS3', strict=False) is None:
    # als folder nog niet bestaat, maak hem dan aan
    client.makedirs('/user/bigdata/HDFS3')    # /user/bigdata/HDFS3 is gelijk aan HDFS3 omdat we hierboven de user als bigdata ingesteld hebben

# bestanden schrijven
client.upload('HDFS3', 'davinci_notebooks.txt')

client.write('HDFS3/test.txt', 'outline_of_science.txt')

HdfsError: Remote path '/user/bigdata/HDFS3/davinci_notebooks.txt' already exists.

### Bekijken van het filesysteem

In [16]:
# met ! gaan we het onderstaande commando op de terminal uitvoeren
# sommige versies van jupyter geven ze de voorkeur aan % ipv !
!hdfs dfs -ls /user/bigdata/HDFS2

Found 1 items
-rw-r--r--   3 root supergroup    1586336 2024-02-27 08:26 /user/bigdata/HDFS2/cli.txt


In [22]:
print(client.list('HDFS3'))
print(client.content('HDFS3/test.txt'))
# deze gaan wij minder gaan gebruiken
print(client.acl_status('HDFS3/davinci_notebooks.txt'))
print(client.checksum('HDFS3/davinci_notebooks.txt'))
print(client.walk('/'))   # dit geeft een generator terug, dit kan je in een for-lus plaatsen
for f in client.walk('/user/bigdata'):
    print(f)

['davinci_notebooks.txt', 'test.txt']
{'directoryCount': 0, 'ecPolicy': 'Replicated', 'fileCount': 1, 'length': 22, 'quota': -1, 'snapshotDirectoryCount': 0, 'snapshotFileCount': 0, 'snapshotLength': 0, 'snapshotSpaceConsumed': 0, 'spaceConsumed': 66, 'spaceQuota': -1, 'typeQuota': {}}
{'entries': [], 'group': 'supergroup', 'owner': 'bigdata', 'permission': '644', 'stickyBit': False}
{'algorithm': 'MD5-of-0MD5-of-512CRC32C', 'bytes': '0000020000000000000000009a18185c475e208de5db172f18473b2b00000000', 'length': 28}
<generator object Client.walk at 0x7f0b51ee3220>
('/user/bigdata', ['ETL', 'HDFS', 'HDFS2', 'HDFS3', 'MLLib', 'MapReduce', 'Oef_MapReduce', 'Oefeningen', 'Project', 'Spark'], [])
('/user/bigdata/ETL', [], ['cars.csv'])
('/user/bigdata/HDFS', [], ['davinci_notebooks.txt'])
('/user/bigdata/HDFS2', [], ['cli.txt'])
('/user/bigdata/HDFS3', [], ['davinci_notebooks.txt', 'test.txt'])
('/user/bigdata/MLLib', ['chihuahua-or-muffin'], [])
('/user/bigdata/MLLib/chihuahua-or-muffin', []

## Uitlezen van het filesysteem

In [30]:
with client.read('HDFS3/test.txt') as reader:
    content = reader.read()
    print(content)

client.download('HDFS3/davinci_notebooks.txt', 'notebooks2.txt')

b'outline_of_science.txt'


'/home/bigdata/workspace2324/Week 02/notebooks2.txt'

### Aanpassen van het filesysteem

In [ ]:
client.set_replication('HDFS3/test.txt', 1)
client.set_permission('HDFS3/davinci_notebooks.txt', 700)

client.rename('HDFS3/test.txt', 'HDFS3/test2.txt')

In [36]:
client.delete('/user/bigdata/HDFS3', recursive=True)
client.delete('/user/bigdata/HDFS2', recursive=True)

True

### Oefening

Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.

In [44]:
from hdfs import InsecureClient
client = InsecureClient("http://localhost:9870", user='bigdata')
map = 'HDFS'
input_files = [ 'davinci_notebooks.txt', 'outline_of_science.txt', 'ulysses.txt']

if client.status(map, strict=False) is None:
    client.makedirs(map)
else:
    for f in client.list(map):
        # extra check om ervoor te zorgen dat startdata niet verwijderd wordt
        if not f.endswith('.txt'):
            client.delete(map + "/" + f, recursive=True)

for f in input_files:
    if client.status(map + "/" + f) is None:
        client.upload(map, f)